<a href="https://colab.research.google.com/github/Iremguel/Fallstudie_Elektrifizierung_der_Logistik/blob/main/L%C3%B6sung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, LpBinary, LpContinuous, lpSum, LpStatus, value

# ==============================================================================
# 1. PARAMETER UND DATEN (Hartcodiert)
# ==============================================================================

# -- Allgemeine Parameter --
DAYS_PER_YEAR = 260.0       # Anforderung: 260 Betriebstage
DIESEL_PRICE_EUR_L = 1.60   # Anforderung: 1,60 Euro/Liter
ELEC_PRICE_WORK = 0.25      # EUR/kWh (Arbeitspreis)
ELEC_PRICE_PEAK = 150.0     # EUR/kW (Jahresleistungsspitze)
ELEC_PRICE_BASE = 1000.0    # EUR/Jahr (Grundgebühr Stromanschluss)

GRID_MAX_POWER_BASE = 500.0   # kW (Bestehender Anschluss)
GRID_EXTENSION_POWER = 500.0  # kW (Mögliche Erweiterung)
GRID_EXTENSION_COST = 10000.0 # EUR/Jahr (Kosten Erweiterung)

MAX_CHARGER_SLOTS = 10  # Maximale Anzahl Ladesäulen am Standort (Bauliches Limit)

# Zeit-Diskretisierung (15 Minuten Slots)
NUM_SLOTS = 96
SLOT_DURATION = 0.25  # h

# Maut (aus PDF/Kontext)
TOLL_DIESEL_EUR_KM = 0.34
TOLL_EV_EUR_KM = 0.0      # Befreit

# -- Daten aus chargers.csv --
# Werte übernommen, keine Einheiten umgerechnet
chargers_data = {
    'Alpitronic-50':  {'capex': 3000,  'opex': 1000, 'power': 50,  'spots': 2},
    'Alpitronic-200': {'capex': 10000, 'opex': 1500, 'power': 200, 'spots': 2},
    'Alpitronic-400': {'capex': 16000, 'opex': 2000, 'power': 400, 'spots': 2}
}

# -- Daten aus diesel_trucks.csv --
diesel_trucks_data = {
    'ActrosL': {
        'capex': 24000,
        'opex': 6000,
        'consumption': 26, # L/100km
        'tax': 556,
        'is_ev': False
    }
}

# -- Daten aus electric_trucks.csv --
# consumption in kWh/100km
electric_trucks_data = {
    'eActros600': {
        'capex': 60000,
        'opex': 6000,
        'consumption': 110, # kWh/100km
        'thg': 1000,
        'power': 400,
        'capacity': 621,
        'is_ev': True
    },
    'eActros400': {
        'capex': 50000,
        'opex': 5000,
        'consumption': 105, # kWh/100km
        'thg': 1000,
        'power': 400,
        'capacity': 414,
        'is_ev': True
    }
}

vehicles_data = {**diesel_trucks_data, **electric_trucks_data}

# -- Daten aus routes.csv --
# Hilfsfunktion: Zeit "HH:MM" -> Slot Index (0..95)
def time_to_slot(t_str):
    h, m = map(int, t_str.split(':'))
    return int((h * 60 + m) / 15)

# Liste aller Routen (hardcodiert aus CSV)
routes_raw = [
    ('t-4', 'Nahverkehr', 250, 150, '06:45', '17:15'),
    ('t-5', 'Nahverkehr', 250, 150, '06:30', '17:00'),
    ('t-6', 'Nahverkehr', 250, 150, '06:00', '16:30'),
    ('s-1', 'Ditzingen', 120, 32, '05:30', '15:30'),
    ('s-2', 'Ditzingen', 120, 32, '06:00', '16:00'),
    ('s-3', 'Ditzingen', 120, 32, '09:00', '16:00'),
    ('s-4', 'Ditzingen', 120, 32, '06:30', '16:30'),
    ('w1', 'Ditzingen', 100, 32, '05:30', '15:30'),
    ('w2', 'Ditzingen', 100, 32, '08:00', '18:00'),
    ('w3', 'Ditzingen', 100, 32, '06:45', '16:45'),
    ('w4', 'Ditzingen', 100, 32, '06:00', '16:00'),
    ('w5', 'Ditzingen', 100, 32, '07:00', '17:00'),
    ('w6', 'Ditzingen', 100, 32, '05:30', '15:30'),
    ('w7', 'Ditzingen', 100, 32, '07:15', '17:15'),
    ('r1', 'MultiStop', 285, 259, '18:00', '22:30'),
    ('r2', 'MultiStop', 250, 220, '16:30', '21:45'),
    ('r3', 'Schramberg', 235, 219, '17:45', '21:30'),
    ('h3', 'Hettingen', 180, 160, '18:45', '22:45'),
    ('h4', 'Hettingen', 180, 160, '18:30', '22:30'),
    ('k1', 'Wendlingen', 275, 235, '16:30', '22:30')
]

routes_data = {}
for rid, name, dist, td, s, e in routes_raw:
    routes_data[rid] = {
        'dist': dist,
        'toll_dist': td,
        'start': time_to_slot(s),
        'end': time_to_slot(e)
    }

# -- Speicher-Daten (aus PDF) --
STORAGE_CAPEX_PWR = 30.0    # EUR/kW
STORAGE_CAPEX_CAP = 350.0   # EUR/kWh
STORAGE_OPEX_PCT = 0.02     # 2% der Investition
STORAGE_EFF = 0.98          # Wirkungsgrad
STORAGE_DOD = 0.975         # Depth of Discharge

# Max Fahrzeuge pro Typ (Sicherheitspuffer: Anzahl Routen)
MAX_VEH_PER_TYPE = len(routes_data)

# ==============================================================================
# 2. OPTIMIERUNGSMODELL
# ==============================================================================

prob = LpProblem("Fleet_Optimization_Case", LpMinimize)

# --- Sets ---
vehicle_types = list(vehicles_data.keys())
ev_types = [v for v in vehicle_types if vehicles_data[v]['is_ev']]
charger_types = list(chargers_data.keys())
route_ids = list(routes_data.keys())
time_slots = range(NUM_SLOTS)

# --- Variablen ---

# Flotte: Wie viele Fahrzeuge welches Typs?
v_exists = {}
for v in vehicle_types:
    for k in range(MAX_VEH_PER_TYPE):
        v_exists[v, k] = LpVariable(f"exists_{v}_{k}", cat=LpBinary)

# Zuweisung: Welche Route fährt welches Fahrzeug?
assign = {}
for r in route_ids:
    for v in vehicle_types:
        for k in range(MAX_VEH_PER_TYPE):
            assign[r, v, k] = LpVariable(f"assign_{r}_{v}_{k}", cat=LpBinary)

# Infrastruktur: Anzahl Charger
n_chargers = {}
for c in charger_types:
    n_chargers[c] = LpVariable(f"num_chargers_{c}", lowBound=0, cat=LpInteger)

grid_extension = LpVariable("grid_extension", cat=LpBinary)
storage_pwr = LpVariable("storage_pwr_kw", lowBound=0)
storage_cap = LpVariable("storage_cap_kwh", lowBound=0)
peak_load = LpVariable("peak_load_kw", lowBound=0)

# Lade-Logik (Nur EVs)
charge_pwr = {}
soc = {}
is_charging = {}
plugged_in = {}

for v in ev_types:
    for k in range(MAX_VEH_PER_TYPE):
        for t in time_slots:
            charge_pwr[v, k, t] = LpVariable(f"chg_pwr_{v}_{k}_{t}", lowBound=0)
            # SOC ist dynamisch, auch Startwert soc[...,0] ist eine Variable
            soc[v, k, t] = LpVariable(f"soc_{v}_{k}_{t}", lowBound=0, upBound=vehicles_data[v]['capacity'])
            plugged_in[v, k, t] = LpVariable(f"plugged_{v}_{k}_{t}", cat=LpBinary)
            for c in charger_types:
                is_charging[v, k, t, c] = LpVariable(f"is_chg_{v}_{k}_{t}_{c}", cat=LpBinary)

# Netzbezug & Speicherfluss
grid_draw = {}
storage_charge = {}
storage_discharge = {}
storage_soc = {}

for t in time_slots:
    grid_draw[t] = LpVariable(f"grid_draw_{t}", lowBound=0)
    storage_charge[t] = LpVariable(f"stor_chg_{t}", lowBound=0)
    storage_discharge[t] = LpVariable(f"stor_dis_{t}", lowBound=0)
    storage_soc[t] = LpVariable(f"stor_soc_{t}", lowBound=0)

# --- Zielfunktion ---

# 1. Flottenkosten (Fix)
cost_fleet_fixed = 0
for v in vehicle_types:
    c_yearly = vehicles_data[v]['capex'] + vehicles_data[v]['opex']
    if vehicles_data[v]['is_ev']:
        c_yearly -= vehicles_data[v]['thg'] # Abzug THG Quote
    else:
        c_yearly += vehicles_data[v]['tax'] # Steuer bei Diesel

    # Summe der existierenden Fahrzeuge * Kosten
    cost_fleet_fixed += lpSum([v_exists[v, k] for k in range(MAX_VEH_PER_TYPE)]) * c_yearly

# 2. Variable Fahrkosten (Diesel & Maut)
# EVs zahlen hier 0 (Strom ist extra), Diesel zahlt Sprit + Maut
cost_var_transport = 0
for r in route_ids:
    for v in vehicle_types:
        if not vehicles_data[v]['is_ev']:
            # Verbrauch (L) = (L/100km / 100) * km
            fuel_amount = (vehicles_data[v]['consumption'] / 100.0) * routes_data[r]['dist']
            fuel_cost = fuel_amount * DIESEL_PRICE_EUR_L
            toll_cost = routes_data[r]['toll_dist'] * TOLL_DIESEL_EUR_KM

            # Kosten pro Jahr = Kosten pro Fahrt * Betriebstage
            cost_var_transport += lpSum([assign[r, v, k] for k in range(MAX_VEH_PER_TYPE)]) * (fuel_cost + toll_cost) * DAYS_PER_YEAR

# 3. Infrastruktur & Speicher
cost_infra = 0
for c in charger_types:
    cost_infra += n_chargers[c] * (chargers_data[c]['capex'] + chargers_data[c]['opex'])

cost_infra += grid_extension * GRID_EXTENSION_COST

storage_invest = (storage_pwr * STORAGE_CAPEX_PWR) + (storage_cap * STORAGE_CAPEX_CAP)
cost_storage = storage_invest * (1 + STORAGE_OPEX_PCT)

# 4. Stromkosten (Energie + Leistung + Grundgebühr)
total_kwh = lpSum([grid_draw[t] for t in time_slots]) * SLOT_DURATION * DAYS_PER_YEAR
cost_energy = total_kwh * ELEC_PRICE_WORK
cost_peak = peak_load * ELEC_PRICE_PEAK
cost_base = ELEC_PRICE_BASE

total_cost = cost_fleet_fixed + cost_var_transport + cost_infra + cost_storage + \
             cost_energy + cost_peak + cost_base

prob += total_cost

# --- Nebenbedingungen ---

# 1. Jede Route muss genau einmal gefahren werden
for r in route_ids:
    prob += lpSum([assign[r, v, k] for v in vehicle_types for k in range(MAX_VEH_PER_TYPE)]) == 1

# 2. Wenn Route zugewiesen, muss Fahrzeug existieren
for r in route_ids:
    for v in vehicle_types:
        for k in range(MAX_VEH_PER_TYPE):
            prob += assign[r, v, k] <= v_exists[v, k]

# 3. Zeitliche Überschneidung (Fahrzeug kann nicht 2 Routen gleichzeitig fahren)
for v in vehicle_types:
    for k in range(MAX_VEH_PER_TYPE):
        for t in time_slots:
            active_routes = []
            for r in route_ids:
                s, e = routes_data[r]['start'], routes_data[r]['end']
                # Prüfen ob Route r im Slot t aktiv ist
                # Normalfall: start <= t < end
                # Über Nacht: start > end, dann t >= start ODER t < end
                is_active = False
                if s < e:
                    if s <= t < e: is_active = True
                else:
                    if t >= s or t < e: is_active = True

                if is_active:
                    active_routes.append(r)

            if active_routes:
                prob += lpSum([assign[r, v, k] for r in active_routes]) <= 1

# 4. EV Logik & SOC
for v in ev_types:
    cons_factor = vehicles_data[v]['consumption'] / 100.0 # kWh pro km

    for k in range(MAX_VEH_PER_TYPE):
        # Zyklische Bedingung: SOC am Ende des Tages = SOC am Anfang
        # Dadurch wird der Start-SOC dynamisch so gewählt, dass er aufgeht.
        prob += soc[v, k, 0] == soc[v, k, NUM_SLOTS-1]

        for t in time_slots:
            next_t = (t + 1) % NUM_SLOTS

            # Verbrauch in diesem Slot berechnen
            energy_used = 0
            is_driving_expr = 0 # Summe aller Zuweisungen, die jetzt fahren

            for r in route_ids:
                s, e = routes_data[r]['start'], routes_data[r]['end']

                # Fährt das Fahrzeug auf Route r in Slot t?
                driving_now = False
                if s < e:
                    if s <= t < e: driving_now = True
                else:
                    if t >= s or t < e: driving_now = True

                if driving_now:
                    # Dauer der Route in Slots
                    dur = e - s if e > s else (NUM_SLOTS - s + e)
                    # Verbrauch pro Slot = Gesamtverbrauch / Dauer
                    c_slot = (routes_data[r]['dist'] * cons_factor) / dur
                    energy_used += assign[r, v, k] * c_slot
                    # Blockiert Ladeanschluss
                    prob += plugged_in[v, k, t] + assign[r, v, k] <= 1

            # SOC Bilanz
            # Neuer SOC = Alter SOC - Verbrauch + Geladen
            prob += soc[v, k, next_t] == soc[v, k, t] - energy_used + (charge_pwr[v, k, t] * SLOT_DURATION)

            # Ladeleistung Limits
            prob += charge_pwr[v, k, t] <= vehicles_data[v]['power'] # Fahrzeug Limit
            prob += charge_pwr[v, k, t] <= plugged_in[v, k, t] * vehicles_data[v]['power'] # Nur wenn plugged

            # Charger Zuweisung
            prob += plugged_in[v, k, t] == lpSum([is_charging[v, k, t, c] for c in charger_types])

            # Limit durch Charger Typ
            for c in charger_types:
                # Wenn an Typ C, dann MaxPower = Typ C Power. Sonst 'unendlich' (bzw. Fzg Limit)
                prob += charge_pwr[v, k, t] <= (is_charging[v, k, t, c] * chargers_data[c]['power']) + (1 - is_charging[v, k, t, c]) * 1000

# 5. Infrastruktur Limits
prob += lpSum([n_chargers[c] for c in charger_types]) <= MAX_CHARGER_SLOTS

for t in time_slots:
    for c in charger_types:
        # Anzahl Fahrzeuge an Charger C <= Anzahl Charger C * Spots
        prob += lpSum([is_charging[v, k, t, c] for v in ev_types for k in range(MAX_VEH_PER_TYPE)]) <= n_chargers[c] * chargers_data[c]['spots']

# 6. Grid & Speicher
for t in time_slots:
    total_ev_load = lpSum([charge_pwr[v, k, t] for v in ev_types for k in range(MAX_VEH_PER_TYPE)])

    # Bilanz am Netzanschlusspunkt
    prob += grid_draw[t] + storage_discharge[t] == total_ev_load + storage_charge[t]

    # Grid Limit
    grid_cap = GRID_MAX_POWER_BASE + (grid_extension * GRID_EXTENSION_POWER)
    prob += grid_draw[t] <= grid_cap

    # Peak Load erfassen
    prob += peak_load >= grid_draw[t]

    # Speicher SOC
    next_t = (t + 1) % NUM_SLOTS
    # SOC Update
    prob += storage_soc[next_t] == storage_soc[t] + (storage_charge[t] * STORAGE_EFF * SLOT_DURATION) - (storage_discharge[t] / STORAGE_EFF * SLOT_DURATION)

    # Speicher Limits
    prob += storage_soc[t] <= storage_cap * STORAGE_DOD
    prob += storage_charge[t] <= storage_pwr
    prob += storage_discharge[t] <= storage_pwr

# Zyklus Speicher
prob += storage_soc[0] == storage_soc[NUM_SLOTS-1]

# ==============================================================================
# 3. LÖSUNG & AUSGABE
# ==============================================================================
solver = pulp.PULP_CBC_CMD(msg=True, timeLimit=300000000000000000000)
prob.solve(solver)

print(f"Status: {LpStatus[prob.status]}")
print(f"Total Annual Cost: {value(prob.objective):,.2f} EUR")

print("\n--- FLOTTEN-MIX ---")
for v in vehicle_types:
    cnt = sum([value(v_exists[v, k]) for k in range(MAX_VEH_PER_TYPE)])
    if cnt > 0.5:
        print(f"  {v}: {int(cnt)}")

print("\n--- LADEINFRASTRUKTUR ---")
for c in charger_types:
    val = value(n_chargers[c])
    if val > 0.5:
        print(f"  {c}: {int(val)} Säulen")

print("\n--- NETZ & SPEICHER ---")
print(f"  Netzerweiterung nötig: {'JA' if value(grid_extension) > 0.5 else 'NEIN'}")
print(f"  Speicher Leistung: {value(storage_pwr):.2f} kW")
print(f"  Speicher Kapazität: {value(storage_cap):.2f} kWh")
print(f"  Peak Load (Netz): {value(peak_load):.2f} kW")

print("\n--- KOSTEN DETAILS (EUR/Jahr) ---")
print(f"  Flotte (Fix): {value(cost_fleet_fixed):,.2f}")
print(f"  Transport (Diesel/Maut): {value(cost_var_transport):,.2f}")
print(f"  Infrastruktur (Chargers + Grid Ext): {value(cost_infra):,.2f}")
print(f"  Speicher (Invest + Opex): {value(cost_storage):,.2f}")
print(f"  Energie (Strom Arbeit): {value(cost_energy):,.2f}")
print(f"  Leistung (Strom Peak): {value(cost_peak):,.2f}")